In [13]:
import pandas as pd
date=input("Enter data: ")
df=pd.read_csv('./Latest_ETM_Data/Final_Ticket_data_2024-07-01.csv')
df.shape

Enter data: 01


(1978353, 32)

In [14]:
all_zonal_centroids = range(1, 2915)

# Create all possible source-destination pairs
all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

# Group and sum passenger counts, then reindex to include all pairs
grouped = df.groupby(['source_zonal_centroid_number', 'destination_zonal_centroid_number'])['passenger_count'].sum()
od_matrix = grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)

# Add a column for the row sums
od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['Alighting'] = od_matrix.sum(axis=0)

od_matrix

destination_zonal_centroid_number,1,2,3,4,5,6,7,8,9,10,...,2906,2907,2908,2909,2910,2911,2912,2913,2914,Boarding
source_zonal_centroid_number,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,6,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,34200
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2912,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2913,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
import pandas as pd

all_zonal_centroids = range(1, 2915)

# Create all possible source-destination pairs
all_pairs = pd.MultiIndex.from_product(
    [all_zonal_centroids, all_zonal_centroids],
    names=['source_zonal_centroid_number', 'destination_zonal_centroid_number']
)

combined_grouped = pd.Series(0, index=all_pairs, dtype='int64')

# Loop through each date and aggregate passenger counts
for day in range(1, 8):  # From 01 to 07
    date = f"2024-07-{day:02d}"
    file_path = f"./Latest_ETM_Data/Final_Ticket_data_{date}.csv"
    
    # Load the daily data
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue
    
    # Group passenger counts for this day's data
    daily_grouped = df.groupby(
        ['source_zonal_centroid_number', 'destination_zonal_centroid_number']
    )['passenger_count'].sum()
    
    # Add to the combined grouped data
    combined_grouped = combined_grouped.add(daily_grouped, fill_value=0)

# Construct the OD matrix from the combined passenger counts
od_matrix = combined_grouped.reindex(all_pairs, fill_value=0).unstack(fill_value=0)

# Add a column for the row sums (Total Passengers)
od_matrix['Total Passengers'] = od_matrix.sum(axis=1)

od_matrix.head(10)

File not found: ./Latest_ETM_Data/Final_Ticket_data_2024-07-02.csv
File not found: ./Latest_ETM_Data/Final_Ticket_data_2024-07-03.csv
File not found: ./Latest_ETM_Data/Final_Ticket_data_2024-07-04.csv
File not found: ./Latest_ETM_Data/Final_Ticket_data_2024-07-05.csv
File not found: ./Latest_ETM_Data/Final_Ticket_data_2024-07-06.csv
File not found: ./Latest_ETM_Data/Final_Ticket_data_2024-07-07.csv


destination_zonal_centroid_number,1,2,3,4,5,6,7,8,9,10,...,2906,2907,2908,2909,2910,2911,2912,2913,2914,Total Passengers
source_zonal_centroid_number,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34200.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,839.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
